# Buidling NLP Models 

In [1]:
import pandas as pd
import numpy as np
import gensim.downloader
import re
from gensim.models import word2vec, phrases
from gensim.parsing.preprocessing import remove_stopwords, strip_punctuation, strip_numeric,\
                    strip_non_alphanum, strip_multiple_whitespaces, strip_short
from textblob import TextBlob, Word
import collections
from numpy import dot
from numpy.linalg import norm
from gensim.test.utils import common_texts
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
import requests
from PIL import Image
from io import BytesIO
from urllib.request import urlopen
import matplotlib.pyplot as plt
import matplotlib.image as mpimg

In [2]:
pd.set_option('display.max_rows', None)
pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_columns', None)

## Load cleaned DF as pickle file

In [4]:
df = pd.read_pickle("../../docker_data/r_cleaned_recipes.pkl")
# # df.head(1)

In [5]:
#----Checking that list formats are preserved
df['ingredients'].iloc[217559][0]

'olive oil'

### "Clean" ingredients to remove common ingredients

In [8]:
#----function to clean words in ingredients

def clean_words(list_of_words):
    
    output_list = []
    for word in list_of_words:
        
        ing = word.lower()
        if word == 'pepper':
            ing = word.replace('pepper', "")
        else:
            ing = word
            
        ing = remove_stopwords(ing)
        ing = strip_numeric(ing)
        
        ing = re.sub(r'\(.*oz.\)|(®)|(.*ed)|(.*ly)|boneless|skinless|\
                    chunks|fresh|large|cook drain|frozen|ground|\
                    |salt|sea salt|sugar|onion|ginger|water|olive oil|oil|flour|\
                    |cloves|clove|all-purpose flour|brown sugar|baking powder|\
                    |baking soda|vanilla|cinnamon|vanilla extract|black pepper|white pepper|\
                    |unsalted butter|butter|salted butter|sour cream| powder|granulated|\
                    |extra virgin|fresh ground black pepper|heavy cream|powdered|\
                    |ground|grounded|ketchup|whole wheat|water|boiling water|warm water|vinegar','',ing).strip(" ")
        
        ing = strip_short(ing,2)
        ing = strip_multiple_whitespaces(ing)
        ing = strip_punctuation(ing)
        ing = strip_non_alphanum(ing)
        if word != 'asparagus':
            ing = (" ".join(TextBlob(ing).words.singularize()))
        output_list.append(ing)
    
    for item in output_list:
        if item == '':
            output_list.remove(item)
    
    return output_list


In [38]:
df.head(1)

,Name,RecipeCategory,Calories,images,cook_time,prep_time,total_time,recipe_servings,tags,instructions,ingredient_quantities,ingredients,ingredients_cleaned,ingred_and_tags
0,Low-Fat Berry Blue Frozen Dessert,Frozen Desserts,170.9,"[https://img.sndimg.com/food/image/upload/w_555,h_416,c_fit,fl_progressive,q_95/v1/img/recipes/38/yueirxmlqaee1h3v3qnm_229%20berry%20blue%20frzn%20dess.jpg, https://img.sndimg.com/food/image/upload/w_555,h_416,c_fit,fl_progressive,q_95/v1/img/recipes/38/afpddhatwzq0b1cdpdat_255%20berry%20blue%20frzn%20dess.jpg, https://img.sndimg.com/food/image/upload/w_555,h_416,c_fit,fl_progressive,q_95/v1/img/recipes/38/uygf9nwmt2sggjcuzilo_228%20berry%20blue%20frzn%20dess.jpg, https://img.sndimg.com/food/image/upload/w_555,h_416,c_fit,fl_progressive,q_95/v1/img/recipes/38/pebmjn2tgsayks2759ba_20140722_202142.jpg, https://img.sndimg.com/food/image/upload/w_555,h_416,c_fit,fl_progressive,q_95/v1/img/recipes/38/picuaeten.jpg, https://img.sndimg.com/food/image/upload/w_555,h_416,c_fit,fl_progressive,q_95/v1/img/recipes/38/pictzvxw5.jpg]",24 Hour(s),45 Minute(s),24 Hour(s) 45 Minute(s),4,"[dessert, low protein, low cholesterol, healthy, free of..., summer, weeknight, freezer, easy]","[Toss 2 cups berries with sugar., Let stand for 45 minutes., Stirring occasionally., Transfer berry-sugar mixture to food processor., Add yogurt and process until smooth., Strain through fine sieve. pour into baking pan (or transfer to ice cream maker and process according to manufacturers' directions). freeze uncovered until edges are solid but centre is soft. transfer to processor and blend until smooth again., Return to pan and freeze until edges are solid., Transfer to processor and blend until smooth again., Fold in remaining 2 cups of blueberries., Pour into plastic mold and freeze overnight. let soften slightly to serve.]","[4, 1/4, 1, 1]","[blueberries, granulated sugar, vanilla yogurt, lemon juice]","[blueberry, yogurt, lemon juice]","[blueberry, yogurt, lemon juice, dessert, low protein, low cholesterol, healthy, free of..., summer, weeknight, freezer, easy]"


In [9]:
#-----Apply clean function and create ingredients_cleaned column
df['ingredients_cleaned'] = df['ingredients'].apply(clean_words).apply(clean_words)

In [46]:
#Turning Name and RecipeCategory to list
def string_to_list(string):
    s = [word.strip() for word in string.split(" ")]
    return s
        
df["name"] = df["Name"].apply(string_to_list)
df['recipe_cat'] = df["RecipeCategory"].apply(string_to_list)

In [75]:
#----Combine tags and ingredients for nlp training
df['ingred_and_tags'] = df['ingredients_cleaned'] + df['tags']

#----combine tags, ingred, name and cat for nlp training
df["name_cat_ingred_tag"] = df['name'] + df['recipe_cat']+ df['ingredients_cleaned'] + df['tags']

#----combine tags, ingred and cat for nlp training
df['cat_ingred_tag'] = df['recipe_cat']+ df['ingredients_cleaned'] + df['tags']

In [76]:
df.head(1)

,Name,RecipeCategory,Calories,images,cook_time,prep_time,total_time,recipe_servings,tags,instructions,ingredient_quantities,ingredients,ingredients_cleaned,ingred_and_tags,name,recipe_cat,name_cat_ingred_tag,cat_ingred_tag
0,Low-Fat Berry Blue Frozen Dessert,Frozen Desserts,170.9,"[https://img.sndimg.com/food/image/upload/w_555,h_416,c_fit,fl_progressive,q_95/v1/img/recipes/38/yueirxmlqaee1h3v3qnm_229%20berry%20blue%20frzn%20dess.jpg, https://img.sndimg.com/food/image/upload/w_555,h_416,c_fit,fl_progressive,q_95/v1/img/recipes/38/afpddhatwzq0b1cdpdat_255%20berry%20blue%20frzn%20dess.jpg, https://img.sndimg.com/food/image/upload/w_555,h_416,c_fit,fl_progressive,q_95/v1/img/recipes/38/uygf9nwmt2sggjcuzilo_228%20berry%20blue%20frzn%20dess.jpg, https://img.sndimg.com/food/image/upload/w_555,h_416,c_fit,fl_progressive,q_95/v1/img/recipes/38/pebmjn2tgsayks2759ba_20140722_202142.jpg, https://img.sndimg.com/food/image/upload/w_555,h_416,c_fit,fl_progressive,q_95/v1/img/recipes/38/picuaeten.jpg, https://img.sndimg.com/food/image/upload/w_555,h_416,c_fit,fl_progressive,q_95/v1/img/recipes/38/pictzvxw5.jpg]",24 Hour(s),45 Minute(s),24 Hour(s) 45 Minute(s),4,"[dessert, low protein, low cholesterol, healthy, free of..., summer, weeknight, freezer, easy]","[Toss 2 cups berries with sugar., Let stand for 45 minutes., Stirring occasionally., Transfer berry-sugar mixture to food processor., Add yogurt and process until smooth., Strain through fine sieve. pour into baking pan (or transfer to ice cream maker and process according to manufacturers' directions). freeze uncovered until edges are solid but centre is soft. transfer to processor and blend until smooth again., Return to pan and freeze until edges are solid., Transfer to processor and blend until smooth again., Fold in remaining 2 cups of blueberries., Pour into plastic mold and freeze overnight. let soften slightly to serve.]","[4, 1/4, 1, 1]","[blueberries, granulated sugar, vanilla yogurt, lemon juice]","[blueberry, yogurt, lemon juice]","[blueberry, yogurt, lemon juice, dessert, low protein, low cholesterol, healthy, free of..., summer, weeknight, freezer, easy]","[Low-Fat, Berry, Blue, Frozen, Dessert]","[Frozen, Desserts]","[Low-Fat, Berry, Blue, Frozen, Dessert, Frozen, Desserts, blueberry, yogurt, lemon juice, dessert, low protein, low cholesterol, healthy, free of..., summer, weeknight, freezer, easy]","[Frozen, Desserts, blueberry, yogurt, lemon juice, dessert, low protein, low cholesterol, healthy, free of..., summer, weeknight, freezer, easy]"


In [81]:
df2 = df.drop(columns=["name", "recipe_cat"])
df2.head(1)

,Name,RecipeCategory,Calories,images,cook_time,prep_time,total_time,recipe_servings,tags,instructions,ingredient_quantities,ingredients,ingredients_cleaned,ingred_and_tags,name_cat_ingred_tag,cat_ingred_tag
0,Low-Fat Berry Blue Frozen Dessert,Frozen Desserts,170.9,"[https://img.sndimg.com/food/image/upload/w_555,h_416,c_fit,fl_progressive,q_95/v1/img/recipes/38/yueirxmlqaee1h3v3qnm_229%20berry%20blue%20frzn%20dess.jpg, https://img.sndimg.com/food/image/upload/w_555,h_416,c_fit,fl_progressive,q_95/v1/img/recipes/38/afpddhatwzq0b1cdpdat_255%20berry%20blue%20frzn%20dess.jpg, https://img.sndimg.com/food/image/upload/w_555,h_416,c_fit,fl_progressive,q_95/v1/img/recipes/38/uygf9nwmt2sggjcuzilo_228%20berry%20blue%20frzn%20dess.jpg, https://img.sndimg.com/food/image/upload/w_555,h_416,c_fit,fl_progressive,q_95/v1/img/recipes/38/pebmjn2tgsayks2759ba_20140722_202142.jpg, https://img.sndimg.com/food/image/upload/w_555,h_416,c_fit,fl_progressive,q_95/v1/img/recipes/38/picuaeten.jpg, https://img.sndimg.com/food/image/upload/w_555,h_416,c_fit,fl_progressive,q_95/v1/img/recipes/38/pictzvxw5.jpg]",24 Hour(s),45 Minute(s),24 Hour(s) 45 Minute(s),4,"[dessert, low protein, low cholesterol, healthy, free of..., summer, weeknight, freezer, easy]","[Toss 2 cups berries with sugar., Let stand for 45 minutes., Stirring occasionally., Transfer berry-sugar mixture to food processor., Add yogurt and process until smooth., Strain through fine sieve. pour into baking pan (or transfer to ice cream maker and process according to manufacturers' directions). freeze uncovered until edges are solid but centre is soft. transfer to processor and blend until smooth again., Return to pan and freeze until edges are solid., Transfer to processor and blend until smooth again., Fold in remaining 2 cups of blueberries., Pour into plastic mold and freeze overnight. let soften slightly to serve.]","[4, 1/4, 1, 1]","[blueberries, granulated sugar, vanilla yogurt, lemon juice]","[blueberry, yogurt, lemon juice]","[blueberry, yogurt, lemon juice, dessert, low protein, low cholesterol, healthy, free of..., summer, weeknight, freezer, easy]","[Low-Fat, Berry, Blue, Frozen, Dessert, Frozen, Desserts, blueberry, yogurt, lemon juice, dessert, low protein, low cholesterol, healthy, free of..., summer, weeknight, freezer, easy]","[Frozen, Desserts, blueberry, yogurt, lemon juice, dessert, low protein, low cholesterol, healthy, free of..., summer, weeknight, freezer, easy]"


### Save df with new columns

In [78]:
# uncomment to save
# df.to_pickle('r_cleaned_recipes_3.pkl')

## Building NLP model for ingredients

### Applying tags to list of cleaned ingredients for each recipe

In [10]:
#tagging ingredients
ingredients_cleaned_documents = [TaggedDocument(doc, [i]) for i, doc in enumerate(df['ingredients_cleaned'])]

### Instantiate model and train

In [11]:
#building nlp model for ingredients

doc_model_i = Doc2Vec(ingredients_cleaned_documents, 
                      vector_size= 50, 
                      alpha = 0.025,
                      min_count= 100, 
                      epochs = 50,
                      window = 10,  
                      dm=0)

doc_model_i.build_vocab(ingredients_cleaned_documents)

doc_model_i.train(ingredients_cleaned_documents, 
                total_examples = doc_model_i.corpus_count, 
                epochs= 100)

### Save trained model to local drive

In [12]:
#uncomment below to save
# doc_model_i.save("r_ingred_d2v.model")

### Vectorise user input of ingredients


In [67]:
#checking similarities for ingredients

input_ingred = "strawberry, chocolate, egg"

tokens_ingred = input_ingred.split(", ")
new_vector_ingred = doc_model_i.infer_vector(tokens_ingred)
most_similar_ingred = doc_model_i.dv.most_similar([new_vector_ingred])
print(most_similar_ingred)
most_similar_ingred_list = [x[0] for x in most_similar_ingred]


[(76859, 0.9607111215591431), (206125, 0.9337906837463379), (24869, 0.929422914981842), (330684, 0.9286863803863525), (228377, 0.9248731136322021), (287431, 0.9216048121452332), (104284, 0.9206739068031311), (460665, 0.9201422929763794), (47190, 0.875518262386322), (452519, 0.8742367029190063)]


### Return top 10 recipes baed on most similar ingredients

In [68]:
recipe_ingred_dict = {}
for i in most_similar_ingred_list:
    recipe_ingred_dict[i] = df.iloc[i].to_dict()

most_similar_ingred_df = pd.DataFrame(recipe_ingred_dict).T.reset_index()

most_similar_ingred_df

,index,Name,RecipeCategory,Calories,images,cook_time,prep_time,total_time,recipe_servings,tags,instructions,ingredient_quantities,ingredients,ingredients_cleaned,ingred_and_tags,name,recipe_cat,name_cat_ingred_tag
0,76859,Grand Marnier Parfait,Bar Cookie,591.8,[Not Provided],20 Minute(s),5 Minute(s),25 Minute(s),Not Provided,"[frozen desserts, dessert, cookie & brownie, european, sweet, mixer, < 30 mins, freezer, small appliance]","[Beat eggs and yolks., Add sugar and liqueur and mix thoroughly., Heat au bain marie while continuously stirring until slightly thicker (don't let it boil!)., Take from heat and whip until thickened (circa 10 minutes)., Whip cream until thick but still slightly fluid., Fold through egg mixture., Pour into form (circa 1 litre) and freeze for circa 5 hours., Remove parfait from freezer 30 minutes before serving., Turn onto a large plate., Serve parfait with shaved chocolate and strawberry slices.""\n.]","[4, 2, 150, 100, 300, 5, 50, 2]","[eggs, sugar, strawberries, chocolate]","[egg, strawberry, chocolate]","[egg, strawberry, chocolate, frozen desserts, dessert, cookie & brownie, european, sweet, mixer, < 30 mins, freezer, small appliance]","[Grand, Marnier, Parfait]","[Bar, Cookie]","[Grand, Marnier, Parfait, Bar, Cookie, egg, strawberry, chocolate, frozen desserts, dessert, cookie & brownie, european, sweet, mixer, < 30 mins, freezer, small appliance]"
1,206125,Chocolate Dipped Strawberries,Dessert,236.1,"[https://img.sndimg.com/food/image/upload/w_555,h_416,c_fit,fl_progressive,q_95/v1/img/recipes/21/50/42/picnroqw2.jpg]",5 Minute(s),20 Minute(s),25 Minute(s),Not Provided,"[strawberry, berries, fruit, spring, summer, microwave, < 30 mins, stove top, easy]","[Rinse strawberries., Drying them with paper towels., Melt chocolate in top of double boiler or in microwave (full power in 30 second intervals., Until chocolate is melted., Stirring often)., Note 1: chocolate should not be too hot - if it gets too hot it will drip on the berries instead of adhering to the berries., Note 2: be careful--over heating chocolate may cause scorching and keep it from blending properly., Insert toothpick into stem end of strawberry and dip 3/4 of berry into chocolate., Leaving green cap and some of the red berry showing., Spear free end of toothpick into a piece of styrofoam or grapefruit to allow strawberry to dry upside down (alternatively dry strawberries on waxed paper)., Note 3: if chocolate becomes too thick to work with reheat chocolate gently., Note 4: after chocolate hardens dip tip of berry into contrasting melted chocolate., After the chocolate hardens discard toothpicks., These berries are best if eaten within 24 hours., For longer storage refrigerate but serve at room temperature.]","[24 -30, 6, na]","[strawberries, chocolate]","[strawberry, chocolate]","[strawberry, chocolate, strawberry, berries, fruit, spring, summer, microwave, < 30 mins, stove top, easy]","[Chocolate, Dipped, Strawberries]",[Dessert],"[Chocolate, Dipped, Strawberries, Dessert, strawberry, chocolate, strawberry, berries, fruit, spring, summer, microwave, < 30 mins, stove top, easy]"
2,24869,Chocolate Covered Strawberries,Dessert,249.4,"[https://img.sndimg.com/food/image/upload/w_555,h_416,c_fit,fl_progressive,q_95/v1/img/recipes/28/38/0/picwysbal.jpg]",20 Minute(s),5 Minute(s),25 Minute(s),2,"[lunch/snacks, strawberry, berries, fruit, canadian, scottish, european, kid friendly, potluck, < 30 mins, beginner cook, stove top, easy, inexpensive]","[Cover a baking sheet with non-stick baking paper., Break up the chocolate and place in a small metal bowl., Place over a saucepan of hot water and heat until melted., Remove from the heat and stir., Do not remove the leaves from the strawberries., Dip into the chocolate., Leaving part of the fruit uncovered., Place on the non-stick baking paper and leave until the chocolate is dry and set.]","[1/2, 3]","[fresh strawberries, chocolate]","[strawberry, chocolate]","[strawberry, chocolate, lunch/sna

## Building NLP model for tags/keywords

### Applying tags to list of tags for each recipe

In [21]:
#tagging keywords
tags_documents = [TaggedDocument(doc, [i]) for i, doc in enumerate(df['tags'])]

### Instatiate model and train

In [22]:
#building nlp model for tags
doc_model_t = Doc2Vec(tags_documents, 
                      vector_size= 50, 
                      alpha = 0.025,
                      min_count= 20, 
                      epochs = 50,
                      window = 10,  
                      dm=0)

doc_model_t.build_vocab(tags_documents)

doc_model_t.train(tags_documents, 
                total_examples = doc_model_t.corpus_count, 
                epochs= 100)

### Save trained model to local drive

In [37]:
#uncomment below to save
# doc_model_t.save("r_tags_d2v.model")

### Vectorise user input of preferences

In [66]:
input_tags = "summer, easy, refresh"

tokens_tags= input_tags.split(", ")
new_vector_tags = doc_model_t.infer_vector(tokens_tags)
most_similar_tags = doc_model_t.dv.most_similar([new_vector_tags], topn= 50)
print(most_similar_tags)
most_similar_tags_list = [x[0] for x in most_similar_tags]

### Return top 10 recipes based on most similar tags

In [65]:
tags_dict = {}
for i in most_similar_tags_list:
    tags_dict[i] = df.iloc[i].to_dict()

most_similar_tags_df = pd.DataFrame(tags_dict).T.reset_index()

most_similar_tags_df

## One model trained on both ingredients and tags

### Applying tags

In [71]:
#tagging 
it_documents = [TaggedDocument(doc, [i]) for i, doc in enumerate(df['ingred_and_tags'])]

### Instatiate model and train

In [72]:
#building nlp model for ingredients

doc_model_both = Doc2Vec(it_documents, 
                      vector_size= 100, 
                      alpha = 0.025,
                      min_count= 100, 
                      epochs = 100,
                      window = 10,  
                      dm=0)

doc_model_both.build_vocab(it_documents)

doc_model_both.train(it_documents, 
                total_examples = doc_model_both.corpus_count, 
                epochs= doc_model_both.epochs)

### Save model

In [73]:
#uncomment below to save
# doc_model_both.save("r_ingredtag_d2v.model") #vector_size = 50, epochs = 100
# doc_model_both.save("r_ingredtag_d2v_2.model") #vector_size = 100, epochs = 100

### Vectorise user inputs

In [30]:
inputs = "chicken, spinach, tomato, < 30 mins, easy"

tokens = inputs.split(", ")
vectors = doc_model_both.infer_vector(tokens)
most_similar_vec = doc_model_both.dv.most_similar([vectors], topn= 10)
# print(most_similar_vec)
most_similar_vec_list = [x[0] for x in most_similar_vec]

### Return top 10 recipes based on highest similarities

In [37]:
vec_dict = {}
for i in most_similar_vec_list:
    vec_dict[i] = df.iloc[i].to_dict()

most_similar_vec_df = pd.DataFrame(vec_dict).T.reset_index()

most_similar_vec_df.head(1)

## One model trained on ingredients, tags, recipe name and categories

### Applying tags

In [51]:
#tagging 
all_documents = [TaggedDocument(doc, [i]) for i, doc in enumerate(df['name_cat_ingred_tag'])]

In [69]:
#building nlp model for all

doc_model_all = Doc2Vec(all_documents, 
                      vector_size= 100, 
                      alpha = 0.025,
                      min_count= 100, 
                      epochs = 100,
                      window = 15,  
                      dm=0)

doc_model_all.build_vocab(all_documents)

doc_model_all.train(all_documents, 
                total_examples = doc_model_all.corpus_count, 
                epochs=doc_model_all.epochs)

### Save model

In [80]:
# uncomment to save
# doc_model_all.save('r_all_d2v.model') #vector_size_50 #epoch 100
# doc_model_all.save('r_all_d2v_2.model') #vector_size_80 #epoch 100
# doc_model_all.save('r_all_d2v_3.model') #vector_size_100 epoch 50
# doc_model_all.save('r_all_d2v_4.model') #vector_size_100 epoch 100

### Vectorise user inputs and return top 5 recipes based on highest similarity score

In [57]:
all_inputs = "chicken, spinach, tomato, < 30 mins, easy"

all_tokens = all_inputs.split(", ")
all_vectors = doc_model_all.infer_vector(tokens)
most_similar_all = doc_model_all.dv.most_similar([all_vectors], topn= 5)
# print(most_similar_all)
most_similar_all_list = [x[0] for x in most_similar_vec]

all_dict = {}
for i in most_similar_all_list:
    all_dict[i] = df.iloc[i].to_dict()

most_similar_vec_df = pd.DataFrame(all_dict).T.reset_index()

most_similar_vec_df.head(1)

[(429460, 0.8286301493644714), (455735, 0.8152256608009338), (72374, 0.7675464153289795), (222894, 0.7531178593635559), (105838, 0.7363920211791992)]


,index,Name,RecipeCategory,Calories,images,cook_time,prep_time,total_time,recipe_servings,tags,instructions,ingredient_quantities,ingredients,ingredients_cleaned,ingred_and_tags,name_cat_ingred_tag
0,429460,Breakfast Burrito Scramble,Breakfast,89.9,[Not Provided],10 Minute(s),10 Minute(s),20 Minute(s),1,"[< 30 mins, easy]","[Combine the eggs., Salt., Pepper., And dillweed., And mix in a bowl. warm olive oil in skillet and add garlic., And onions to warmed skillet to sautee. once golden., Add the eggs and scramble to satisfaction. after the eggs are scrambled to your satisfaction., Add the chopped tomatoes and cooked meat if desired. place in low carb or whole-wheat tortilla. this is a very high protein., Filling., And very portable breakfast for those days when you’re on the go.]","[3, 1/4, 1, 1, na, 1, 1, 1/4]","[onion, tomatoes, garlic clove, spinach, salt, black pepper, ground chicken, olive oil]","[tomato, garlic, spinach, chicken]","[tomato, garlic, spinach, chicken, < 30 mins, easy]","[Breakfast, Burrito, Scramble, Breakfast, tomato, garlic, spinach, chicken, < 30 mins, easy]"


## One model trained on ingredients, tags and categories

In [4]:
df = pd.read_pickle("../../docker_data/final_cleaned_recipes_dataset.pkl")
df.head(1)

,Name,RecipeCategory,Calories,images,cook_time,prep_time,total_time,recipe_servings,tags,instructions,ingredient_quantities,ingredients,ingredients_cleaned,ingred_and_tags,name_cat_ingred_tag,cat_ingred_tag
0,Low-Fat Berry Blue Frozen Dessert,Frozen Desserts,170.9,"[https://img.sndimg.com/food/image/upload/w_555,h_416,c_fit,fl_progressive,q_95/v1/img/recipes/38/yueirxmlqaee1h3v3qnm_229%20berry%20blue%20frzn%20dess.jpg, https://img.sndimg.com/food/image/upload/w_555,h_416,c_fit,fl_progressive,q_95/v1/img/recipes/38/afpddhatwzq0b1cdpdat_255%20berry%20blue%20frzn%20dess.jpg, https://img.sndimg.com/food/image/upload/w_555,h_416,c_fit,fl_progressive,q_95/v1/img/recipes/38/uygf9nwmt2sggjcuzilo_228%20berry%20blue%20frzn%20dess.jpg, https://img.sndimg.com/food/image/upload/w_555,h_416,c_fit,fl_progressive,q_95/v1/img/recipes/38/pebmjn2tgsayks2759ba_20140722_202142.jpg, https://img.sndimg.com/food/image/upload/w_555,h_416,c_fit,fl_progressive,q_95/v1/img/recipes/38/picuaeten.jpg, https://img.sndimg.com/food/image/upload/w_555,h_416,c_fit,fl_progressive,q_95/v1/img/recipes/38/pictzvxw5.jpg]",24 Hour(s),45 Minute(s),24 Hour(s) 45 Minute(s),4,"[dessert, low protein, low cholesterol, healthy, free of..., summer, weeknight, freezer, easy]","[Toss 2 cups berries with sugar., Let stand for 45 minutes., Stirring occasionally., Transfer berry-sugar mixture to food processor., Add yogurt and process until smooth., Strain through fine sieve. pour into baking pan (or transfer to ice cream maker and process according to manufacturers' directions). freeze uncovered until edges are solid but centre is soft. transfer to processor and blend until smooth again., Return to pan and freeze until edges are solid., Transfer to processor and blend until smooth again., Fold in remaining 2 cups of blueberries., Pour into plastic mold and freeze overnight. let soften slightly to serve.]","[4, 1/4, 1, 1]","[blueberries, granulated sugar, vanilla yogurt, lemon juice]","[blueberry, yogurt, lemon juice]","[blueberry, yogurt, lemon juice, dessert, low protein, low cholesterol, healthy, free of..., summer, weeknight, freezer, easy]","[Low-Fat, Berry, Blue, Frozen, Dessert, Frozen, Desserts, blueberry, yogurt, lemon juice, dessert, low protein, low cholesterol, healthy, free of..., summer, weeknight, freezer, easy]","[Frozen, Desserts, blueberry, yogurt, lemon juice, dessert, low protein, low cholesterol, healthy, free of..., summer, weeknight, freezer, easy]"


In [5]:
#tagging 
cit_documents = [TaggedDocument(doc, [i]) for i, doc in enumerate(df['cat_ingred_tag'])]

In [6]:
#building nlp model for cit

doc_model_cit = Doc2Vec(cit_documents, 
                      vector_size= 100, 
                      alpha = 0.025,
                      min_count= 100, 
                      epochs = 100,
                      window = 15,  
                      dm=0)

doc_model_cit.build_vocab(cit_documents)

doc_model_cit.train(cit_documents, 
                total_examples = doc_model_cit.corpus_count, 
                epochs=doc_model_cit.epochs)

In [9]:
doc_model_cit.save('nlp_model_cit_model') #vector_size_100 epoch 100

In [87]:
all_inputs = "chicken, spinach, tomato, < 30 mins, easy"

all_tokens = all_inputs.split(", ")
all_vectors = doc_model_all.infer_vector(tokens)
most_similar_all = doc_model_all.dv.most_similar([all_vectors], topn= 5)
# print(most_similar_all)
most_similar_all_list = [x[0] for x in most_similar_vec]

all_dict = {}
for i in most_similar_all_list:
    all_dict[i] = df.iloc[i].to_dict()

most_similar_vec_df = pd.DataFrame(all_dict).T.reset_index()
most_similar_vec_df.head(1)

,index,Name,RecipeCategory,Calories,images,cook_time,prep_time,total_time,recipe_servings,tags,instructions,ingredient_quantities,ingredients,ingredients_cleaned,ingred_and_tags,name,recipe_cat,name_cat_ingred_tag,cat_ingred_tag
0,429460,Breakfast Burrito Scramble,Breakfast,89.9,[Not Provided],10 Minute(s),10 Minute(s),20 Minute(s),1,"[< 30 mins, easy]","[Combine the eggs., Salt., Pepper., And dillweed., And mix in a bowl. warm olive oil in skillet and add garlic., And onions to warmed skillet to sautee. once golden., Add the eggs and scramble to satisfaction. after the eggs are scrambled to your satisfaction., Add the chopped tomatoes and cooked meat if desired. place in low carb or whole-wheat tortilla. this is a very high protein., Filling., And very portable breakfast for those days when you’re on the go.]","[3, 1/4, 1, 1, na, 1, 1, 1/4]","[onion, tomatoes, garlic clove, spinach, salt, black pepper, ground chicken, olive oil]","[tomato, garlic, spinach, chicken]","[tomato, garlic, spinach, chicken, < 30 mins, easy]","[Breakfast, Burrito, Scramble]",[Breakfast],"[Breakfast, Burrito, Scramble, Breakfast, tomato, garlic, spinach, chicken, < 30 mins, easy]","[Breakfast, tomato, garlic, spinach, chicken, < 30 mins, easy]"
